# <u> Sentiment Analysis On Twitter Posts <u>

In [304]:
import pandas as pd

In [305]:
data_df = pd.read_csv("sentiment_100K.csv", quotechar='"', encoding= "ISO-8859-1")
data_df.shape

(100000, 5)

In [ ]:
data_df=data_df.drop('Topic',1)
data_df=data_df.drop('TweetId',1)
data_df=data_df.drop('TweetDate',1)
data_df.shape

In [ ]:
data_df.columns = ["Sentiment","TweetText"]
positive_sentiment = data_df[data_df['Sentiment']=='positive']
negative_sentiment = data_df[data_df['Sentiment']=='negative']
print(positive_sentiment.shape)
print(negative_sentiment.shape)

In [ ]:
val = 10000
frames = [positive_sentiment[:val], negative_sentiment[:val]]
data_df = pd.concat(frames)
data_df.shape

In [306]:
data_df.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText
0,Topic Placeholder,negative,2254622816,Sat Jun 20 10:01:59 PDT 2009,@maaricela hahahahah nothing ugh my eye is al...
1,Topic Placeholder,negative,2254623107,Sat Jun 20 10:02:00 PDT 2009,People are tweeting advice on how to combat te...
2,Topic Placeholder,negative,2254623203,Sat Jun 20 10:02:01 PDT 2009,"can't go 2 the anberlin show 2nite, tickets so..."
3,Topic Placeholder,negative,2254623331,Sat Jun 20 10:02:01 PDT 2009,Woke up with lots of wisdom teeth drama..Gotta...
4,Topic Placeholder,negative,2254623412,Sat Jun 20 10:02:02 PDT 2009,@tom_teel Not yet dude... Gonna have to buy a ...


In [307]:
data_df.Sentiment.value_counts()

negative    100000
Name: Sentiment, dtype: int64

In [308]:
import numpy as np
print("Average # words per post: ",np.mean([len(s.split(" ")) for s in data_df.TweetText]))

('Average # words per post: ', 14.77861)


In [309]:
test_set_length = int(0.3*(len(data_df)))
training_set_length = int((len(data_df)) - test_set_length)
print(test_set_length)
print(training_set_length)
print(training_set_length +test_set_length)

30000
70000
100000


In [310]:
training_set = data_df[0:training_set_length]
test_set = data_df[training_set_length:]
print("training_set shape: ",training_set.shape)
print("test_set shape: ",test_set.shape)

('training_set shape: ', (70000, 5))
('test_set shape: ', (30000, 5))


In [311]:
del data_df
#import gc
#gc.collect()

1128

In [312]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [313]:
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

In [314]:
import re as regex , nltk
def tokenize(text):
    # remove non letters
    text = regex.sub("[^a-zA-Z]", " ", text)
    # tokenize
    tokens = nltk.word_tokenize(text)
    # stem
    stems = stem_tokens(tokens, stemmer)
    return stems

In [315]:
from sklearn.feature_extraction.text import CountVectorizer 
vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize, # tokenize is a user defined function declared above
    lowercase = True,
    stop_words = 'english',
    #max_features = 200
)

In [316]:
features_matrix = vectorizer.fit_transform(training_set.TweetText.tolist() + test_set.TweetText.tolist())
#features_matrix = vectorizer.fit_transform(train_data_df.Text.tolist() + test_data_df.Text.tolist())

In [317]:
#features = features_matrix.toarray()
#features.shape
#del features_matrix

In [318]:
vocab = vectorizer.get_feature_names()
#print(vocab)
#del vocab

In [319]:
#dist = np.sum(features, axis=0)
#for tag, count in zip(vocab, dist):
#    print count, tag

In [320]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test  = train_test_split(
        features_matrix, 
        training_set.Sentiment.tolist() + test_set.Sentiment.tolist(),#data_df.Sentiment
        test_size=0.30, 
        random_state=3)

In [321]:
del training_set
del test_set

In [322]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression()
log_model = log_model.fit(X=X_train, y=y_train)
y_pred = log_model.predict(X_test)

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: u'negative'

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
NB_model = gnb.fit(X=X_train, y=y_train)
NB_predictions = NB_model.predict(X_test)

In [ ]:
print(classification_report(y_test, NB_predictions))

In [ ]:
from sklearn.svm import SVC
svm_clf = SVC()
svm_model = svm_clf.fit(X=X_train, y=y_train) 
svm_predictions = svm_model.predict(X_test)

In [ ]:
print(classification_report(y_test, svm_predictions))

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf_predictions = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, clf_predictions))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=40)
clf = clf.fit(X_train, y_train)
clf_predictions = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, clf_predictions))

In [ ]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(X_train, y_train)
clf_predictions = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, clf_predictions))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()
clf.fit(X_train, y_train)
clf_predictions = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, clf_predictions))